In [75]:
#Import relevant libraries
import io
import itertools
import sklearn.metrics
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
# import datetime
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tensorboard.plugins.hparams import api as hp

In [76]:
#Dataset encoding explaination.
#image labelling - 0=glasses/sunglasses. 1= trousers/jeans. 3= shoes

In [77]:
#Import the datasets and preprocess

data_train = np.load(r'Full Dataset/primary categories - Train.npz')
data_validation = np.load(r'Full Dataset/primary categories - Validation.npz')
data_Test = np.load(r'Full Dataset/primary categories - Test.npz')


#t extract the arrays from dataset into input(images) and target(labels)
images_train = data_train['images']
labels_train = data_train['labels']

images_val = data_train['images']
labels_val = data_train['labels']


images_test = data_train['images']
labels_test = data_train['labels']

#Pixel-wise normalization of the training, validation and testing data
images_train = images_train/255.0
images_val  = images_val/255.0
images_test = images_test/255.0







In [78]:
#Define the hyperparameters

BATCH_SIZE = 64
EPOCHS = 1

#Define the hyperparamets to tune and the variations we want to test.
HP_FILTER_SIZE = hp.HParam('filter_size', hp.Discrete([3,5,7]))
HP_FILTER_NUM = hp.HParam('filter_num', hp.Discrete(['32','64','96', '128']))

METRIC_ACCURACY = 'accuracy'

#Log the hyperparameter with the file writer
with tf.summary.create_file_writer('Logs/Model 1/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams=[HP_FILTER_SIZE, HP_FILTER_NUM], 
        metrics= [hp.Metric(METRIC_ACCURACY, display_name='Accuracy')]
    )



In [79]:
#CREATE THE MODEL AND TRAIN IT

#Stop the model from overfitting ie whenever the validation loss increases
#the code tells the model to stop when the val_loss starts to increase in two subsequent epochs

#directory to store the TensorBoard logs  
session_num = 0
log_dir = f"Logs/Model 1/fit/run-{session_num}"
summary_writer = tf.summary.create_file_writer(log_dir +'/cm')

def train_model(hparams,session_num):
    
    
#Create and train the model


    
    model = Sequential([
                Conv2D(32, 3, activation= 'relu', input_shape=(120,90,3)),
                MaxPooling2D(pool_size=(2,2)),
                Conv2D(32, 3, activation= 'relu'),
                MaxPooling2D(pool_size=(2,2)),
                Flatten(),
                Dense(3)
    ])


#describe the loss function
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

#Compile the model
    model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])





# Create a Confusion Matrix

    def make_predictions(images_val):
        predictions = model.predict(images_val)
        return predictions



        
    def confusion_matrix_callback(epoch,log=None):

        val_predict_raw= make_predictions(images_val)
        val_predict = np.argmax(val_predict_raw, axis =1)



        cm = confusion_matrix(labels_val, val_predict)
        
        
         
# log the confusion matrix as a heatmap

    def log_confusion_matrix_to_tensorboard(cm,class_labels):
     


        plt.figure(figsize=(6, 6))
        sns.set(font_scale=1.2)  # Adjust the font size
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", square=True,
                    xticklabels=class_labels,
                    yticklabels=class_labels)


        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.title('Confusion Matrix')
         
        
        with summary_writer.as_default():

            tf.summary.image('Confusion Matrix', [plt.gcf()], step=step)
            log_confusion_matrix_to_tensorboard(cm, class_labels= ["Glasses/SUnglassses", "Trousers/Jeans", "Shoes"],
            step=0) 
            
      



    cm_callback = tf.keras.callbacks.LambdaCallback(on_train_end=confusion_matrix_callback)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


    early_stopping = EarlyStopping(
        monitor= 'val_loss',
        mode = 'auto',
        min_delta = 0,
        patience = 2,
        verbose = 0,
        restore_best_weights = True
    )


#Train the model
    model.fit(
        images_train,
        labels_train,
        epochs = EPOCHS,
        batch_size = BATCH_SIZE,
        callbacks = [tensorboard_callback, cm_callback, early_stopping],
        validation_data = (images_val, labels_val),
        verbose =2
    )

    _,accuracy = model.evaluate(images_val, labels_val)

#     model.save(f"Logs/Model 1/fit/run-{session_num}")


    return accuracy







   
   

In [80]:
#Log the confusion matrix and the hyperparameter to the TensorBoard

def run(log_dir,hparams,session_num):
    
    summary_writer = tf.summary.create_file_writer(log_dir)
    
    with summary_writer.as_default():
        hp.hparams(hparams) #record the values used in this trial
        accuracy = train_model(hparams, session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
       
        

       
        
        
        








In [81]:
        
#Train the model with the different hyperparameters
session_num = 1
for filter_size in HP_FILTER_SIZE.domain.values:
    for filter_num in HP_FILTER_NUM.domain.values:
        hparams ={
        HP_FILTER_SIZE: filter_size,
        HP_FILTER_NUM : filter_num
        }
        
        run_name = "run-%d" % session_num
        print('---Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('Logs/Model 1/hparam_tuning/' + run_name, hparams, session_num)
        
        session_num += 1

---Starting trial: run-1
{'filter_size': 3, 'filter_num': '128'}
203/203 - 132s - loss: 0.0750 - accuracy: 0.9747 - val_loss: 0.0099 - val_accuracy: 0.9990 - 132s/epoch - 649ms/step
406/406 [==============================] - 25s 61ms/step - loss: 0.0099 - accuracy: 0.9990
---Starting trial: run-2
{'filter_size': 3, 'filter_num': '32'}
203/203 - 128s - loss: 0.1117 - accuracy: 0.9677 - val_loss: 0.0124 - val_accuracy: 0.9988 - 128s/epoch - 629ms/step
406/406 [==============================] - 27s 66ms/step - loss: 0.0124 - accuracy: 0.9988
---Starting trial: run-3
{'filter_size': 3, 'filter_num': '64'}
203/203 - 134s - loss: 0.0926 - accuracy: 0.9746 - val_loss: 0.0107 - val_accuracy: 0.9987 - 134s/epoch - 659ms/step
406/406 [==============================] - 24s 60ms/step - loss: 0.0107 - accuracy: 0.9987
---Starting trial: run-4
{'filter_size': 3, 'filter_num': '96'}
203/203 - 107s - loss: 0.0905 - accuracy: 0.9718 - val_loss: 0.0122 - val_accuracy: 0.9990 - 107s/epoch - 528ms/step
40

In [82]:
%load_ext tensorboard
%tensorboard --logdir 'Logs/Model 1/hparam_tuning'


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 8096), started 0:42:41 ago. (Use '!kill 8096' to kill it.)

In [74]:
%load_ext tensorboard
%tensorboard --logdir 'Logs/Model 1/fit'

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
